# 9.2 Build a ReAct Agent from Scratch
# 9.2 手写 ReAct Agent

本笔记本将从零开始手写一个 ReAct Agent，不使用任何框架。

**预估成本**: ~$0.05 (使用 GPT-4o-mini)

**安装依赖**:
```bash
pip install openai requests
```

In [ ]:
!pip install openai requests -q

In [ ]:
import os
import re
import requests
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️  警告: 未设置 OPENAI_API_KEY")
else:
    print("✓ API key 已设置")

## 第 1 步：DefineTools
<!-- 第 1 步：定义工具 -->

In [ ]:
def calculator(expression: str) -> str:
    """计算数学表达式"""
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

def wikipedia_search(query: str) -> str:
    """搜索维基百科"""
    try:
        url = "https://en.wikipedia.org/api/rest_v1/page/summary/" + query
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data.get("extract", "No summary found")
        return "Not found"
    except Exception as e:
        return f"Error: {str(e)}"

TOOLS = {
    "calculator": calculator,
    "search": wikipedia_search,
}

print("✓ 工具已注册:", list(TOOLS.keys()))

## 第 2 步：构建 ReAct 提示词

In [ ]:
REACT_PROMPT_TEMPLATE = """You run in a loop of Thought, Action, Observation.
At the end of the loop you output a Final Answer.

Use Thought to describe your reasoning about the question.
Use Action to run one of the available actions.
Observation will be the result of running those actions.

Available actions:
- calculator[expression]: Calculate a math expression. Example: calculator[25 * 4 + 10]
- search[query]: Search Wikipedia. Example: search[Python programming language]
- finish[answer]: Return the final answer and stop. Example: finish[The answer is 42]

Example session:
Question: What is the population of the capital of France?
Thought 1: I need to first find out what the capital of France is.
Action 1: search[capital of France]
Observation 1: Paris is the capital and most populous city of France.
Thought 2: Now I know the capital is Paris. I need to find its population.
Action 2: search[population of Paris]
Observation 2: As of 2023, the population of Paris is approximately 2.2 million.
Thought 3: I have the answer now.
Action 3: finish[The population of Paris is approximately 2.2 million]

Now it's your turn:
Question: {question}
Thought 1:"""

def build_prompt(question: str) -> str:
    return REACT_PROMPT_TEMPLATE.format(question=question)

print("✓ ReAct 提示词模板已定义")

## 第 3 步：解析 Action

In [ ]:
def parse_action(text: str) -> tuple:
    """解析 LLM 输出中的 Action"""
    action_pattern = r"Action \d+: (\w+)\[(.*?)\]"
    match = re.search(action_pattern, text)
    
    if match:
        tool_name = match.group(1)
        tool_input = match.group(2)
        return tool_name, tool_input
    
    return None, None

# Test
test_text = "Action 1: calculator[25 * 4]"
tool, input_val = parse_action(test_text)
print(f"✓ 解析测试: {tool}[{input_val}]")

## 第 4 步：实现 ReAct 循环

In [ ]:
def react_agent(question: str, max_steps: int = 10, verbose: bool = True) -> str:
    """ReAct Agent 主循环"""
    prompt = build_prompt(question)
    
    if verbose:
        print("=" * 80)
        print(f"Question: {question}")
        print("=" * 80)
    
    for step in range(1, max_steps + 1):
        # LLM 生成推理和行动
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        
        llm_output = response.choices[0].message.content
        
        if verbose:
            print(f"\n{llm_output}")
        
        # Parse Action
        tool_name, tool_input = parse_action(llm_output)
        
        if tool_name is None:
            return "Error: Failed to parse action"
        
        # 检查是否结束
        if tool_name == "finish":
            if verbose:
                print("\n" + "=" * 80)
                print(f"Final Answer: {tool_input}")
                print("=" * 80)
            return tool_input
        
        # Execute tool
        if tool_name not in TOOLS:
            observation = f"Error: Unknown tool '{tool_name}'"
        else:
            observation = TOOLS[tool_name](tool_input)
        
        if verbose:
            print(f"Observation {step}: {observation}")
        
        # 更新 prompt
        prompt += llm_output
        prompt += f"\nObservation {step}: {observation}\n"
        prompt += f"Thought {step + 1}:"
    
    return "Error: Max steps reached without finishing"

print("✓ ReAct Agent 已定义")

## 第 5 步：Test Agent
<!-- 第 5 步：测试 Agent -->

In [ ]:
# Test 1: 简单计算
answer1 = react_agent("What is (123 + 456) * 789?")

In [ ]:
# Test 2: 搜索问题
answer2 = react_agent("Who created Python programming language?")

In [ ]:
# Test 3: 多步推理
answer3 = react_agent("What is the population of the birthplace of the creator of Python?")

## 第 6 步：添加统计和错误处理

In [ ]:
def react_agent_safe(question: str, max_steps: int = 10, verbose: bool = True) -> dict:
    """带错误处理和统计的 ReAct Agent"""
    prompt = build_prompt(question)
    history = []
    total_tokens = 0
    
    for step in range(1, max_steps + 1):
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
            )
            
            llm_output = response.choices[0].message.content
            total_tokens += response.usage.total_tokens
            
            history.append({
                "step": step,
                "thought_and_action": llm_output,
            })
            
            tool_name, tool_input = parse_action(llm_output)
            
            if tool_name is None:
                return {
                    "status": "error",
                    "message": "Failed to parse action",
                    "history": history,
                }
            
            if tool_name == "finish":
                return {
                    "status": "success",
                    "answer": tool_input,
                    "steps": step,
                    "total_tokens": total_tokens,
                    "history": history,
                }
            
            observation = TOOLS.get(tool_name, lambda x: f"Unknown tool: {tool_name}")(tool_input)
            history[-1]["observation"] = observation
            
            prompt += llm_output
            prompt += f"\nObservation {step}: {observation}\n"
            prompt += f"Thought {step + 1}:"
            
        except Exception as e:
            return {
                "status": "error",
                "message": str(e),
                "history": history,
            }
    
    return {
        "status": "timeout",
        "message": f"Max steps ({max_steps}) reached",
        "history": history,
    }

# Test安全版本
result = react_agent_safe("What is 25 * 4 + 10?", verbose=False)
print(f"Status: {result['status']}")
print(f"Answer: {result.get('answer', 'N/A')}")
print(f"Steps: {result.get('steps', 'N/A')}")
print(f"Tokens: {result.get('total_tokens', 'N/A')}")

## 总结

你已经手写了一个完整的 ReAct Agent！

### 核心组件
1. **工具注册表**: 定义可用的工具
2. **ReAct 提示词**: 引导 LLM 遵循 ReAct 模式
3. **Action 解析器**: 从 LLM 输出提取工具调用
4. **主循环**: Think → Act → Observe 循环

### 关键点
- ✓ Prompt 工程是关键
- ✓ 循环直到 finish
- ✓ 每步都传递完整上下文
- ✓ 需要错误处理

### 下一步

手写 Agent 代码量大、容易出错。下一节学习用框架简化开发！